Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [13]:
batch_size = 17
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

#   layer2_weights = tf.Variable(tf.truncated_normal(
#       [patch_size, patch_size, depth, depth], stddev=0.1))
#   layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
# Try reducing layer 2's output depth from 16 to 13
  new_l2_depth = 13
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, new_l2_depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[new_l2_depth]))

#   layer3_weights = tf.Variable(tf.truncated_normal(
#       [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
#   layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * new_l2_depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print('hidden.get_shape()', hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print('hidden.get_shape()', hidden.get_shape())
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape.get_shape()', reshape.get_shape())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden.get_shape()', hidden.get_shape())
    zzz =  tf.matmul(hidden, layer4_weights) + layer4_biases
    print('zzz.get_shape()', zzz.get_shape())    
    return zzz
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print('------------ valid ----------------')
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  print('------------ test ----------------')
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

hidden.get_shape() (17, 14, 14, 16)
hidden.get_shape() (17, 7, 7, 13)
reshape.get_shape() (17, 637)
hidden.get_shape() (17, 64)
zzz.get_shape() (17, 10)
------------ valid ----------------
hidden.get_shape() (10000, 14, 14, 16)
hidden.get_shape() (10000, 7, 7, 13)
reshape.get_shape() (10000, 637)
hidden.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)
------------ test ----------------
hidden.get_shape() (10000, 14, 14, 16)
hidden.get_shape() (10000, 7, 7, 13)
reshape.get_shape() (10000, 637)
hidden.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)


In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.447521
Minibatch accuracy: 5.9%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.946312
Minibatch accuracy: 11.8%
Validation accuracy: 27.2%
Minibatch loss at step 100: 1.926946
Minibatch accuracy: 47.1%
Validation accuracy: 69.6%
Minibatch loss at step 150: 0.488214
Minibatch accuracy: 82.4%
Validation accuracy: 74.7%
Minibatch loss at step 200: 0.808814
Minibatch accuracy: 70.6%
Validation accuracy: 76.2%
Minibatch loss at step 250: 0.442063
Minibatch accuracy: 82.4%
Validation accuracy: 76.5%
Minibatch loss at step 300: 0.814514
Minibatch accuracy: 76.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.699234
Minibatch accuracy: 70.6%
Validation accuracy: 78.7%
Minibatch loss at step 400: 0.329125
Minibatch accuracy: 88.2%
Validation accuracy: 78.5%
Minibatch loss at step 450: 1.316770
Minibatch accuracy: 64.7%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.640530
Minibatch accuracy: 82.4%
Validation accuracy: 78.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [17]:
batch_size = 17
patch_size = 5
depth = 16
num_hidden = 64
max_pool_k_size = 2
max_pool_stride = 2

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

# Try reducing layer 2's output depth from 16 to 15
  new_l2_depth = 15
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, new_l2_depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[new_l2_depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * new_l2_depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # Layer 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 2
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 3
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape.get_shape()', reshape.get_shape())
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden3.get_shape()', hidden3.get_shape())
    
    # Layer 4
    zzz =  tf.matmul(hidden3, layer4_weights) + layer4_biases
    print('zzz.get_shape()', zzz.get_shape())    
    return zzz
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print('------------ valid ----------------')
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  print('------------ test ----------------')
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

pool.get_shape() (17, 14, 14, 16)
pool.get_shape() (17, 7, 7, 15)
reshape.get_shape() (17, 735)
hidden3.get_shape() (17, 64)
zzz.get_shape() (17, 10)
------------ valid ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)
------------ test ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)


In [18]:
# Copy paste without changes
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.356928
Minibatch accuracy: 11.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.243181
Minibatch accuracy: 5.9%
Validation accuracy: 23.8%
Minibatch loss at step 100: 2.552833
Minibatch accuracy: 23.5%
Validation accuracy: 51.2%
Minibatch loss at step 150: 0.612642
Minibatch accuracy: 64.7%
Validation accuracy: 64.6%
Minibatch loss at step 200: 0.796955
Minibatch accuracy: 82.4%
Validation accuracy: 76.4%
Minibatch loss at step 250: 0.297566
Minibatch accuracy: 94.1%
Validation accuracy: 73.2%
Minibatch loss at step 300: 1.004367
Minibatch accuracy: 64.7%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.843267
Minibatch accuracy: 70.6%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.345751
Minibatch accuracy: 82.4%
Validation accuracy: 80.1%
Minibatch loss at step 450: 1.369456
Minibatch accuracy: 58.8%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.619528
Minibatch accuracy: 82.4%
Validation accuracy: 80.6%
Mi

### Let's try the above but do maxpool->relu in the first two layers, instead of relu->maxpool.

In [21]:
batch_size = 17
patch_size = 5
depth = 16
num_hidden = 64
max_pool_k_size = 2
max_pool_stride = 2

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

# Try reducing layer 2's output depth from 16 to 15
  new_l2_depth = 15
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, new_l2_depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[new_l2_depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * new_l2_depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # Layer 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME') + layer1_biases
    hidden = tf.nn.relu(pool)    
    print('hidden.get_shape()', hidden.get_shape())
    
    # Layer 2
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME') + layer2_biases
    hidden = tf.nn.relu(pool)
    print('hidden.get_shape()', pool.get_shape())
    
    # Layer 3
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape.get_shape()', reshape.get_shape())
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden3.get_shape()', hidden3.get_shape())
    
    # Layer 4
    zzz =  tf.matmul(hidden3, layer4_weights) + layer4_biases
    print('zzz.get_shape()', zzz.get_shape())    
    return zzz
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print('------------ valid ----------------')
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  print('------------ test ----------------')
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

hidden.get_shape() (17, 14, 14, 16)
hidden.get_shape() (17, 7, 7, 15)
reshape.get_shape() (17, 735)
hidden3.get_shape() (17, 64)
zzz.get_shape() (17, 10)
------------ valid ----------------
hidden.get_shape() (10000, 14, 14, 16)
hidden.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)
------------ test ----------------
hidden.get_shape() (10000, 14, 14, 16)
hidden.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)


In [22]:
# Copy paste without changes
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.690093
Minibatch accuracy: 11.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.801433
Minibatch accuracy: 41.2%
Validation accuracy: 48.0%
Minibatch loss at step 100: 1.872351
Minibatch accuracy: 52.9%
Validation accuracy: 68.5%
Minibatch loss at step 150: 0.439874
Minibatch accuracy: 94.1%
Validation accuracy: 75.4%
Minibatch loss at step 200: 0.743897
Minibatch accuracy: 70.6%
Validation accuracy: 78.7%
Minibatch loss at step 250: 0.270107
Minibatch accuracy: 94.1%
Validation accuracy: 75.9%
Minibatch loss at step 300: 0.854449
Minibatch accuracy: 76.5%
Validation accuracy: 81.0%
Minibatch loss at step 350: 0.659820
Minibatch accuracy: 76.5%
Validation accuracy: 81.2%
Minibatch loss at step 400: 0.344306
Minibatch accuracy: 82.4%
Validation accuracy: 81.7%
Minibatch loss at step 450: 1.411956
Minibatch accuracy: 76.5%
Validation accuracy: 81.7%
Minibatch loss at step 500: 0.651391
Minibatch accuracy: 82.4%
Validation accuracy: 82.2%
M

Ok so looks like `Maxpool(Relu(x)) ~= Relu(Maxpool(x))` as in https://www.quora.com/In-Convolutional-Nets-CNN-isnt-the-maxpool-layer-and-ReLU-layer-redundant

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [23]:
# Only add learning rate decay here. This uses Maxpool(relu(x)).

batch_size = 17
patch_size = 5
depth = 16
num_hidden = 64
max_pool_k_size = 2
max_pool_stride = 2
# Try reducing layer 2's output depth from 16 to 15
new_l2_depth = 15

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, new_l2_depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[new_l2_depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * new_l2_depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # Layer 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 2
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 3
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape.get_shape()', reshape.get_shape())
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden3.get_shape()', hidden3.get_shape())
    
    # Layer 4
    zzz =  tf.matmul(hidden3, layer4_weights) + layer4_biases
    print('zzz.get_shape()', zzz.get_shape())    
    return zzz
  
  # Training computation.
  logits = model(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print('------------ valid ----------------')
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  print('------------ test ----------------')
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

pool.get_shape() (17, 14, 14, 16)
pool.get_shape() (17, 7, 7, 15)
reshape.get_shape() (17, 735)
hidden3.get_shape() (17, 64)
zzz.get_shape() (17, 10)
------------ valid ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)
------------ test ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)


In [24]:
# Copy paste without changes
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.373566
Minibatch accuracy: 11.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.160220
Minibatch accuracy: 29.4%
Validation accuracy: 10.1%
Minibatch loss at step 100: 2.029316
Minibatch accuracy: 29.4%
Validation accuracy: 37.7%
Minibatch loss at step 150: 0.541580
Minibatch accuracy: 88.2%
Validation accuracy: 67.1%
Minibatch loss at step 200: 0.861508
Minibatch accuracy: 64.7%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.537598
Minibatch accuracy: 82.4%
Validation accuracy: 75.9%
Minibatch loss at step 300: 0.961596
Minibatch accuracy: 70.6%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.630805
Minibatch accuracy: 76.5%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.217603
Minibatch accuracy: 88.2%
Validation accuracy: 79.7%
Minibatch loss at step 450: 1.402925
Minibatch accuracy: 58.8%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.681782
Minibatch accuracy: 82.4%
Validation accuracy: 80.7%
M

No difference with learning rate decay. let's try dropout

In [25]:
# Dropout. No learning rate decay.

batch_size = 17
patch_size = 5
depth = 16
num_hidden = 64
max_pool_k_size = 2
max_pool_stride = 2
# Try reducing layer 2's output depth from 16 to 15
new_l2_depth = 15

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, new_l2_depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[new_l2_depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * new_l2_depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  # @param do_dropout - should only be True during training, not validation/testing.
  def model(data, do_dropout):
    # Layer 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 2
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, 
                          ksize=[1, max_pool_k_size, max_pool_k_size, 1], 
                          strides=[1, max_pool_stride, max_pool_stride, 1],
                          padding='SAME')
    if do_dropout:
        pool = tf.nn.dropout(pool, 0.5, seed=100)
    print('pool.get_shape()', pool.get_shape())
    
    # Layer 3
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape.get_shape()', reshape.get_shape())
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    if do_dropout:
        hidden3 = tf.nn.dropout(hidden3, 0.5, seed=100)
    print('hidden3.get_shape()', hidden3.get_shape())
    
    # Layer 4
    zzz =  tf.matmul(hidden3, layer4_weights) + layer4_biases
    print('zzz.get_shape()', zzz.get_shape())    
    return zzz
  
  # Training computation.
  logits = model(tf_train_dataset, do_dropout=True)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print('------------ valid ----------------')
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, do_dropout=False))
  print('------------ test ----------------')
  test_prediction = tf.nn.softmax(model(tf_test_dataset, do_dropout=False))

pool.get_shape() (17, 14, 14, 16)
pool.get_shape() (17, 7, 7, 15)
reshape.get_shape() (17, 735)
hidden3.get_shape() (17, 64)
zzz.get_shape() (17, 10)
------------ valid ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)
------------ test ----------------
pool.get_shape() (10000, 14, 14, 16)
pool.get_shape() (10000, 7, 7, 15)
reshape.get_shape() (10000, 735)
hidden3.get_shape() (10000, 64)
zzz.get_shape() (10000, 10)


In [26]:
# Copy paste without changes
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.363930
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.351591
Minibatch accuracy: 11.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 2.278338
Minibatch accuracy: 17.6%
Validation accuracy: 14.2%
Minibatch loss at step 150: 2.163979
Minibatch accuracy: 5.9%
Validation accuracy: 23.0%
Minibatch loss at step 200: 2.004266
Minibatch accuracy: 29.4%
Validation accuracy: 45.3%
Minibatch loss at step 250: 1.822989
Minibatch accuracy: 35.3%
Validation accuracy: 61.0%
Minibatch loss at step 300: 1.742248
Minibatch accuracy: 47.1%
Validation accuracy: 67.7%
Minibatch loss at step 350: 1.412679
Minibatch accuracy: 52.9%
Validation accuracy: 69.3%
Minibatch loss at step 400: 0.765377
Minibatch accuracy: 70.6%
Validation accuracy: 72.2%
Minibatch loss at step 450: 2.025581
Minibatch accuracy: 35.3%
Validation accuracy: 71.5%
Minibatch loss at step 500: 1.556892
Minibatch accuracy: 52.9%
Validation accuracy: 75.8%
Min

Slightly worse with dropout.

In [28]:
# TODO: Adding another layer or changing depths should give bigger changes here.